In [1]:
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import math
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [17]:

class IntelliGoal(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(s):
        super(IntelliGoal, s).__init__()
        
        s.screen_width = 1240
        s.screen_height = 800
        s.border_width = 10
        s.screen = None
        
        s.goal_radius = 20
        s.player_radius = 10
        
        s.player_speed = 1
        
        
        # State information
        s.player_x = 0
        s.player_y = 0
        s.goal_x = 0
        s.goal_y = 0
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        s.reward = 0
        
        s.done = False
        s.trucated = False
        s.truncation_step_limit = 300
        s.truncation_step_counter = 0
        
        
        # State and Action spaces
        s.action_space = spaces.Discrete(5) # 0: No movement, 1: Up, 2: Down, 3: Left, 4: Right
        # observation space = [player_x, player_y, goal_x, goal_y]
        s.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0]), 
            high=np.array([s.screen_width, s.screen_height, s.screen_width, s.screen_height]), 
            dtype=np.int32
        )
    
    def seed(s, seed=None):
        if seed is not None and (seed < 0 or seed >= 2**32):
            raise ValueError("Seed must be between 0 and 2**32 - 1")
        s.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]
        
    def reset(s, seed=0):
        s.seed(seed)
        
        s.player_x = s.screen_width // 2
        s.player_y = s.screen_height // 2
        
        s.goal_x, s.goal_y = s.respawn_goal()
        
        s.reward = 0
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        
        
        s.done = False
        s.trucated = False
        s.truncation_step_counter = 0
        
        return s.state, {}
    
    def respawn_goal(s):
        s.goal_x = s.player_x + random.randint(-50, 50)
        s.goal_y = s.player_y + random.randint(-50, 50)
        
        # clip the goal to the screen
        offset = s.border_width + s.goal_radius
        s.goal_x = min( max(s.goal_x, offset), s.screen_width - offset)
        s.goal_y = min( max(s.goal_y, offset), s.screen_height - offset)
        
        pygame.time.wait(300)
        
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        
        return s.goal_x, s.goal_y
        
    def step(s, action):
        #################
        #do more here?
        #################
        
        s.do_action(action)
        s.calculate_reward()
        
        s.truncation_step_counter += 1
        if s.truncation_step_counter >= s.truncation_step_limit:
            s.reward -= 0.4
            s.trucated = True
            s.done = True

        if s.reward < -1:
            s.reward = -1
            s.trucated = False
        
        if s.reward > 1:
            s.reward = 1
        
        
        return s.state, s.reward, s.done, s.trucated, {}
        
   
    def do_action(s, action):
        if not s.action_space.contains(action):
            print("Invalid Action")
        
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        
        s.player_x += (action == 4)*s.player_speed - (action == 3)*s.player_speed 
        s.player_y += (action == 2)*s.player_speed - (action == 1)*s.player_speed
        
    def calculate_reward(s):
        # Distance between player and goal
        distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)

        # Check if the red dot touches the border
        if s.player_x not in range(s.border_width, s.screen_width - s.border_width) or s.player_y not in range(s.border_width, s.screen_height - s.border_width):
            s.player_x, s.player_y = s.screen_width // 2, s.screen_height // 2

        # Check if the goal is reached within a certain range
        if distance < s.goal_radius + s.player_radius:
            s.goal_x, s.goal_y = s.respawn_goal()
            s.reward += 1
            s.done = True  
    
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        
        # Reward the agent for getting closer to the goal
        if s.prev_distance > distance:
            s.reward += 0.010
        if s.prev_distance < distance:
            s.reward -= 0.015
        
        # Penalize the agent for taking too long
        s.reward -= 0.0001
     
    
    def keyboard_input(s):
        keys = pygame.key.get_pressed()
        # 1: Up, 2: Down, 3: Left, 4: Right
        if keys[pygame.K_UP]:
            return 1
        if keys[pygame.K_DOWN]:
            return 2
        if keys[pygame.K_LEFT]:
            return 3
        if keys[pygame.K_RIGHT]:
            return 4
        return 0
        
    
    def render(s, mode = 'human'):
        if s.screen == None:
            pygame.init()
            s.screen = pygame.display.set_mode((s.screen_width, s.screen_height))
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        s.player_x, s.player_y, s.goal_x, s.goal_y = s.state
        
        s.screen.fill((0, 0, 0))
        RED = (255, 0, 0)
        BLUE = (0, 0, 255)
        BORDER_COLOR = (255, 255, 0)
        
        # Draw the border
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, 0, s.screen_width, s.border_width))
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, 0, s.border_width, s.screen_height))
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, s.screen_height - s.border_width, s.screen_width, s.border_width))
        pygame.draw.rect(s.screen, BORDER_COLOR, (s.screen_width - s.border_width, 0, s.border_width, s.screen_height))

        # Draw the red dot
        pygame.draw.circle(s.screen, RED, (s.player_x, s.player_y), 10)

        # Draw the blue dot
        pygame.draw.circle(s.screen, BLUE, (s.goal_x, s.goal_y), s.goal_radius)
        
        #Display Reward on pygame screen
        font = pygame.font.Font(None, 36)
        text = font.render("Reward: " + str(round(s.reward, 5)), True, (255, 255, 255))
        s.screen.blit(text, (10, 10))
        
        pygame.display.update()
    
    def close(s):
        pygame.quit()
        
        

In [3]:
# Test Env with keyboard input actions
test_env = IntelliGoal()
test_env.reset(0)
done = False
i = 1

# while True:
#     try:
#         test_env.render()
#         action = test_env.keyboard_input()
#         state, reward, done, truncated, _i = test_env.step(action)
#         if done or truncated:
#             print("Reward at step - ", i, " : ", round(reward, 5))
#             i += 1
#             test_env.reset(0)
#     except:
#         print("Environment Closed")
#         break

# while True:
#     test_env.render()
#     action = test_env.keyboard_input()
#     state, reward, done, truncated, _i = test_env.step(action)
#     if done or truncated:
#         print("Reward at step - ", i, " : ", round(reward, 5))
#         i += 1
#         test_env.reset(0)   

test_env.close()

# PPO Learning Algorithm

In [1]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [5]:
import torch
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU - ", torch.cuda.get_device_name(0))

Using GPU -  NVIDIA RTX A500 Laptop GPU


In [21]:
env = IntelliGoal()
check_env(env)

In [32]:
def make_env():
    env = IntelliGoal()
    return env

num_envs = 2
take_logs = False

if take_logs:
    log_dir = "./logs/ppo_intelligoal_a3_explore-more"
else:
    log_dir = None
      
if num_envs == 1:
    env = DummyVecEnv([make_env] * num_envs)
else:
    env = SubprocVecEnv([make_env] * num_envs)


# model = PPO("MlpPolicy", env, verbose=1, device=device)

# Define DQN model with specific parameters
model = DQN("MlpPolicy", env, verbose=1, 
            buffer_size=10000, 
            learning_rate=0.0005, 
            batch_size=32, 
            tau=1.0, 
            gamma=0.99, 
            train_freq=4, 
            gradient_steps=1, 
            exploration_fraction=0.1, 
            exploration_final_eps=0.01, 
            target_update_interval=1000, 
            max_grad_norm=10, 
            tensorboard_log="./dqn_intelligoal_tensorboard/")

Using cuda device

In [33]:
save_path = "./models/intelli_goal_a6"

# 5. Trying smaller batch size 

In [34]:
# Start New training session
# save_path = "./models/intelli_goal_a2"
# # set teh goal to spawn close - +_ 50 pixels initially

if os.path.exists(save_path +".zip"):
    #append number to the file name
    i = save_path[-1]
    i = int(i) + 1
    save_path = save_path[:-1] + str(i)
    print("Model already exists. Saving new one as - ", save_path)
    
try:
    model.learn(total_timesteps=9999999999, reset_num_timesteps=False, progress_bar=False)
    model.save(save_path)
except KeyboardInterrupt:
    print("Training Interrupted")
    model.save(save_path)

-----------------------------
| time/              |      |
|    fps             | 132  |
|    iterations      | 1    |
|    time_elapsed    | 61   |
|    total_timesteps | 8192 |
-----------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 107         |
|    iterations           | 2           |
|    time_elapsed         | 152         |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.020578865 |
|    clip_fraction        | 0.049       |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.59       |
|    explained_variance   | -0.00346    |
|    learning_rate        | 0.0004      |
|    loss                 | 4.87        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00379    |
|    value_loss           | 10.8        |
-----------------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 102         |
|    iterations           | 3           |
|    time_elapsed         | 239         |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.017659927 |
|    clip_fraction        | 0.0342      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.58       |
|    explained_variance   | 0.0207      |
|    learning_rate        | 0.0004      |
|    loss                 | 5.7         |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.00239    |
|    value_loss           | 15.4        |
-----------------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 99          |
|    iterations           | 4           |
|    time_elapsed         | 330         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.015950857 |
|    clip_fraction        | 0.0151      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.58       |
|    explained_variance   | -6.81e-05   |
|    learning_rate        | 0.0004      |
|    loss                 | 10.8        |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.00114    |
|    value_loss           | 21.2        |
-----------------------------------------

----------------------------------------
| time/                   |            |
|    fps                  | 96         |
|    iterations           | 5          |
|    time_elapsed         | 424        |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.00481885 |
|    clip_fraction        | 0          |
|    clip_range           | 0.3        |
|    entropy_loss         | -1.58      |
|    explained_variance   | 2.26e-06   |
|    learning_rate        | 0.0004     |
|    loss                 | 10.5       |
|    n_updates            | 40         |
|    policy_gradient_loss | -0.000639  |
|    value_loss           | 23.9       |
----------------------------------------

...

-----------------------------------------
| time/                   |             |
|    fps                  | 107         |
|    iterations           | 14          |
|    time_elapsed         | 1069        |
|    total_timesteps      | 114688      |
| train/                  |             |
|    approx_kl            | 0.012911004 |
|    clip_fraction        | 0.0415      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.57       |
|    explained_variance   | 0.00849     |
|    learning_rate        | 0.0004      |
|    loss                 | 22          |
|    n_updates            | 130         |
|    policy_gradient_loss | -0.000233   |
|    value_loss           | 48.7        |
-----------------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 101         |
|    iterations           | 15          |
|    time_elapsed         | 1213        |
|    total_timesteps      | 122880      |
| train/                  |             |
|    approx_kl            | 0.020741679 |
|    clip_fraction        | 0.0217      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.59       |
|    explained_variance   | 0.00331     |
|    learning_rate        | 0.0004      |
|    loss                 | 27.7        |
|    n_updates            | 140         |
|    policy_gradient_loss | 0.000368    |
|    value_loss           | 45          |
-----------------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 100         |
|    iterations           | 16          |
|    time_elapsed         | 1300        |
|    total_timesteps      | 131072      |
| train/                  |             |
|    approx_kl            | 0.020927133 |
|    clip_fraction        | 0.0457      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.59       |
|    explained_variance   | 0.000153    |
|    learning_rate        | 0.0004      |
|    loss                 | 12.3        |
|    n_updates            | 150         |
|    policy_gradient_loss | -0.00145    |
|    value_loss           | 46          |
-----------------------------------------

----------------------------------------
| time/                   |            |
|    fps                  | 101        |
|    iterations           | 17         |
|    time_elapsed         | 1373       |
|    total_timesteps      | 139264     |
| train/                  |            |
|    approx_kl            | 0.02808524 |
|    clip_fraction        | 0.0395     |
|    clip_range           | 0.3        |
|    entropy_loss         | -1.57      |
|    explained_variance   | -0.000393  |
|    learning_rate        | 0.0004     |
|    loss                 | 30.1       |
|    n_updates            | 160        |
|    policy_gradient_loss | -0.00198   |
|    value_loss           | 35.6       |
----------------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 96          |
|    iterations           | 18          |
|    time_elapsed         | 1533        |
|    total_timesteps      | 147456      |
| train/                  |             |
|    approx_kl            | 0.023561908 |
|    clip_fraction        | 0.0147      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.57       |
|    explained_variance   | 1.07e-06    |
|    learning_rate        | 0.0004      |
|    loss                 | 28.5        |
|    n_updates            | 170         |
|    policy_gradient_loss | -5.59e-05   |
|    value_loss           | 49.9        |
-----------------------------------------

In [27]:
# Load and resume training from saved model
try:
    model.load(save_path)
    model.learn(total_timesteps=100000000, reset_num_timesteps=False, progress_bar=True)
    model.save(save_path)
except KeyboardInterrupt:
    print("Training Interrupted")
    print("Saving Model")
    model.save(save_path)

Output()

-------------------------------
| time/              |        |
|    fps             | 92     |
|    iterations      | 1      |
|    time_elapsed    | 88     |
|    total_timesteps | 321209 |
-------------------------------


----------------------------------------
| time/                   |            |
|    fps                  | 76         |
|    iterations           | 2          |
|    time_elapsed         | 214        |
|    total_timesteps      | 329401     |
| train/                  |            |
|    approx_kl            | 0.02335565 |
|    clip_fraction        | 0.0141     |
|    clip_range           | 0.3        |
|    entropy_loss         | -1.6       |
|    explained_variance   | 0          |
|    learning_rate        | 0.0004     |
|    loss                 | 46.2       |
|    n_updates            | 100        |
|    policy_gradient_loss | -0.000357  |
|    value_loss           | 157        |
----------------------------------------


Training Interrupted

Saving Model

In [ ]:
# Evaluate the model
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=1)
print(f"Mean Reward: {mean_reward}, Std Reward: {std_reward}")


Mean Reward: 0.48489999771118164, Std Reward: 0.0


In [13]:
from stable_baselines3.common.vec_env import DummyVecEnv
save_path = "./models/intelli_goal_a3"

In [20]:

def make_env():
    def _init():
        env = IntelliGoal()
        env.render_mode = 'human'
        return env
    return _init

# Wrap your environment in a DummyVecEnv for compatibility
env = DummyVecEnv([make_env()])

model = PPO.load(save_path, env=env, device=device)

def run_model():
    obs = env.reset()  # This will now be in the correct batch format
    done = False
    while True:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, info = env.step(action)
        env.render()
        if done:
            print("Episode Finished with Reward - ", rewards[0])
            obs = env.reset()

try:
    run_model()
except:
    print("Environment Closed")


Episode Finished with Reward -  [-1.]
Episode Finished with Reward -  [-0.43]
Episode Finished with Reward -  [0.9849]
Episode Finished with Reward -  [-0.43]
Episode Finished with Reward -  [1.]
Episode Finished with Reward -  [-1.]
Episode Finished with Reward -  [1.]
Episode Finished with Reward -  [1.]
Episode Finished with Reward -  [-0.43]
Episode Finished with Reward -  [-0.43]
Episode Finished with Reward -  [1.]
Environment Closed


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
